# Sinh Knowledge Graph

In [6]:
!pip install -q -U transformers accelerate bitsandbytes
!pip install -q networkx tqdm pyyaml
!pip install flash-attn --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [7]:
import os
import json
import re
import time
import yaml
import torch
import gc
import pandas as pd
import networkx as nx
from typing import List, Tuple, Union
from tqdm import tqdm
from networkx.readwrite import json_graph
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

## 1. Model Configuration

### 1.1. Model definition

In [8]:
import torch
import gc
from typing import List
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

class LLM:
    def __init__(
        self,
        model_id: str = "Qwen/Qwen2.5-3B-Instruct",
        use_quantization: bool = False # Trên A100 với model < 14B thì để False cho nhanh
    ):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_id = model_id

        print(f">> 📥 Loading Model: {model_id}...")
        print(f">> 🚀 Hardware Optimization: A100 Mode (Flash Attention 2 + bfloat16)")

        # Cấu hình load model
        load_kwargs = {
            "device_map": "auto",
            "trust_remote_code": True,
            "attn_implementation": "flash_attention_2", # Tăng tốc cực mạnh trên A100
            "torch_dtype": torch.bfloat16,              # Chuẩn tốt nhất cho A100
        }

        # Chỉ bật 4-bit nếu model quá lớn hoặc muốn tiết kiệm VRAM
        if use_quantization:
            print(">> ⚠️ Quantization Enabled (Slower but saves VRAM)")
            load_kwargs["quantization_config"] = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16
            )

        try:
            # 1. Load Tokenizer TRƯỚC (Sửa lỗi logic cũ)
            self.tokenizer = AutoTokenizer.from_pretrained(model_id)

            # 2. Cấu hình Padding (Bắt buộc cho Batch Inference)
            self.tokenizer.padding_side = "left"  # Decoder-only model phải pad bên trái
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # 3. Load Model
            self.model = AutoModelForCausalLM.from_pretrained(
                model_id,
                **load_kwargs
            )
            print(">> ✅ Model Loaded Successfully!")

            # Warmup nhẹ
            if self.device == "cuda":
                print(f">> 💾 VRAM Used: {torch.cuda.memory_allocated() / 1024**3:.2f} GiB")

        except Exception as e:
            raise RuntimeError(f"❌ Failed to load model: {e}")

    def generate_response(self, input_text: str, system_prompt: str = "You are a helpful AI assistant.") -> str:
        """Sinh văn bản cho 1 input duy nhất (Dùng để test)"""
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": input_text}
        ]

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.device)

        with torch.no_grad():
            generated_ids = self.model.generate(
                **model_inputs,
                max_new_tokens=512,
                temperature=0.1,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.05
            )

        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        return self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    def generate_batch(self, inputs: List[str], system_prompt: str) -> List[str]:
        """
        Sinh văn bản cho nhiều input cùng lúc (Tối ưu cho A100)
        """
        if not inputs: return []

        # Tạo prompt cho cả batch
        prompts = []
        for text in inputs:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": text}
            ]
            prompts.append(
                self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
            )

        # Tokenize & Padding
        model_inputs = self.tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,       # Padding tự động
            truncation=True
        ).to(self.device)

        with torch.no_grad():
            generated_ids = self.model.generate(
                **model_inputs,
                max_new_tokens=1024,
                temperature=0.15,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.05
            )

        # Cắt bỏ phần input (prompt) khỏi output để lấy câu trả lời
        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        return self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    def unload_model(self):
        print(f">> 🗑️ Unloading model {self.model_id}...")
        if hasattr(self, 'model'): del self.model
        if hasattr(self, 'tokenizer'): del self.tokenizer
        gc.collect()
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        print(">> ✅ Memory cleared!")

### 1.2. Model usage

In [9]:
import time
import torch

def example():
    print('=' * 60)
    print('🚀 A100 PERFORMANCE TEST: Qwen-2.5-3B (Bfloat16 + Flash Attention)')
    print('=' * 60)

    # ---------------------------------------------------------
    # 1. LOAD MODEL
    # ---------------------------------------------------------
    print('\n[1/4] Loading Model...')
    start_load = time.time()

    # Khởi tạo class LLM (Lưu ý: đảm bảo bạn đã chạy cell định nghĩa class LLM ở trên)
    generator = LLM(model_id="Qwen/Qwen2.5-3B-Instruct", use_quantization=False)

    end_load = time.time()
    print(f'✅ Model Loaded in: {end_load - start_load:.2f}s')
    if torch.cuda.is_available():
        print(f'💾 VRAM Allocation: {torch.cuda.memory_allocated()/1024**3:.2f} GB')

    # ---------------------------------------------------------
    # 2. SINGLE INFERENCE TEST (Kiểm tra logic)
    # ---------------------------------------------------------
    print('\n[2/4] Testing Single Inference (Normal Chat)...')
    print('-' * 40)

    user_input = "Giải thích ngắn gọn định luật bảo toàn năng lượng."

    start_single = time.time()
    result = generator.generate_response(user_input)
    end_single = time.time()

    print(f"❓ Input:  {user_input}")
    print(f"💡 Output: {result}")
    print(f"⏱️ Time:   {end_single - start_single:.2f}s")

    # ---------------------------------------------------------
    # 3. BATCH INFERENCE TEST (Kiểm tra tốc độ A100)
    # ---------------------------------------------------------
    print('\n[3/4] Testing Batch Inference (High Throughput)...')
    print("👉 Kịch bản: Xử lý song song 5 câu hỏi cùng lúc.")
    print('-' * 40)

    batch_prompts = [
        "Thủ đô của Pháp là gì?",
        "Viết công thức tính diện tích hình tròn.",
        "Dịch 'Hello world' sang tiếng Việt.",
        "Liệt kê 3 màu cơ bản.",
        "Năm 2024 là năm con gì?"
    ]

    start_batch = time.time()
    # Gọi hàm generate_batch mới thêm vào
    batch_results = generator.generate_batch(batch_prompts, system_prompt="Bạn là trợ lý ngắn gọn.")
    end_batch = time.time()

    total_time = end_batch - start_batch

    for i, (inp, out) in enumerate(zip(batch_prompts, batch_results)):
        print(f"  [{i+1}] Q: {inp} | A: {out.strip()}")

    print('-' * 40)
    print(f"⏱️ Total Batch Time: {total_time:.2f}s (cho {len(batch_prompts)} câu)")
    print(f"⚡ Average Speed:    {total_time / len(batch_prompts):.2f}s/câu")
    print(">> Nhận xét: Batch size càng lớn, thời gian trung bình/câu sẽ càng giảm trên A100.")

    # ---------------------------------------------------------
    # 4. CLEANUP
    # ---------------------------------------------------------
    print('\n[4/4] Cleanup...')
    generator.unload_model()
    print('=' * 60)
    print('DONE.')

In [10]:
example()

🚀 A100 PERFORMANCE TEST: Qwen-2.5-3B (Bfloat16 + Flash Attention)

[1/4] Loading Model...
>> 📥 Loading Model: Qwen/Qwen2.5-3B-Instruct...
>> 🚀 Hardware Optimization: A100 Mode (Flash Attention 2 + bfloat16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

>> ✅ Model Loaded Successfully!
>> 💾 VRAM Used: 5.75 GiB
✅ Model Loaded in: 4.51s
💾 VRAM Allocation: 5.75 GB

[2/4] Testing Single Inference (Normal Chat)...
----------------------------------------
❓ Input:  Giải thích ngắn gọn định luật bảo toàn năng lượng.
💡 Output: Định luật bảo toàn năng lượng là một nguyên tắc cơ bản trong vật lý, cho biết rằng tổng lượng năng lượng trong một hệ thống không thay đổi theo thời gian, chỉ chuyển từ một dạng thành dạng khác. Nghĩa là, năng lượng không bao giờ được tạo ra hay mất đi mà chỉ chuyển đổi từ một dạng sang dạng khác. Đây là một trong những nguyên tắc cơ bản nhất trong vật lý và có ứng dụng rộng rãi trong nhiều lĩnh vực khoa học và công nghệ.
⏱️ Time:   7.91s

[3/4] Testing Batch Inference (High Throughput)...
👉 Kịch bản: Xử lý song song 5 câu hỏi cùng lúc.
----------------------------------------
  [1] Q: Thủ đô của Pháp là gì? | A: Thủ đô của Pháp là Paris.
  [2] Q: Viết công thức tính diện tích hình tròn. | A: Diện tích hình tròn là πr², 

## 2. Graph Generator

### 2.1. Generator Setup

In [11]:
def load_configs(path):
    print(f">> Reading config from: {path}")
    try:
        with open(path, "r", encoding="utf-8") as f:
            config = yaml.safe_load(f)
        return config
    except FileNotFoundError:
        print(f"⚠️ Không tìm thấy file config tại: {path}. Dùng default.")
        return {}
    except Exception as e:
        print(f"❌ Lỗi YAML: {e}")
        return {}

In [12]:
class GraphConstructor:
    E2E_PROMPT = (
        "Bạn là một chuyên gia về Xây dựng Đồ thị Tri thức (Knowledge Graph) Tiếng Việt.\n"
        "Nhiệm vụ: Trích xuất các bộ ba ngữ nghĩa (Chủ thể, Quan hệ, Đối tượng) từ văn bản được cung cấp.\n\n"
        "### QUY TẮC BẮT BUỘC:\n"
        "1. **NGÔN NGỮ**: Toàn bộ kết quả (relation, type, description...) PHẢI LÀ TIẾNG VIỆT. TUYỆT ĐỐI KHÔNG DỊCH sang tiếng Anh.\n"
        "2. **Quan hệ (Relation)**: Phải trích xuất cụm động từ hoặc giới từ xuất hiện trong văn bản gốc (Ví dụ: 'là thủ đô của', 'tọa lạc tại', 'đầu tư vào').\n"
        "3. **Thực thể (Head/Tail)**: Giữ nguyên tên riêng và danh từ gốc.\n"
        "4. **Đồng tham chiếu**: Thay thế các đại từ ('nó', 'ông ấy') bằng tên thực thể cụ thể nếu biết.\n\n"
        "5. **SỬA LỖI INPUT NẾU CẦN**: Một số Input có thể sẽ sai chính tả, hãy sửa nếu cần thiết. LƯU Ý: CHỈ THAY ĐỔI WORD-BY-WORD."
        "### ĐỊNH DẠNG OUTPUT (JSON ONLY):\n"
        "Chỉ trả về JSON object theo cấu trúc sau, không thêm lời dẫn:\n"
        "{\n"
        "  \"graph\": [\n"
        "    {\n"
        "      \"head\": \"Tên thực thể 1\",\n"
        "      \"head_props\": {\"type\": \"Loại thực thể (VD: Quốc gia, Người)\", \"description\": \"Mô tả ngắn gọn bằng tiếng Việt\"},\n"
        "      \"relation\": \"động từ quan hệ (tiếng Việt)\",\n"
        "      \"relation_props\": {\"time\": \"thời gian (nếu có)\", \"loc\": \"địa điểm (nếu có)\"},\n"
        "      \"tail\": \"Tên thực thể 2\",\n"
        "      \"tail_props\": {\"type\": \"Loại thực thể\", \"description\": \"Mô tả\"}\n"
        "    }\n"
        "  ]\n"
        "}"
    )

    def __init__(self, config_path: str = None, llm_instance = None):
        """
        Modified: Thêm tham số llm_instance để truyền model đã load bên ngoài vào (tránh load lại).
        Nếu không truyền, giữ logic cũ: tự load config và tự khởi tạo LLM.
        """
        if llm_instance:
            self.llm = llm_instance
        else:
            # Logic cũ: Tự load nếu không truyền instance vào
            # (Giả lập load_configs để code chạy được độc lập trong ví dụ này)
            # full_config = load_configs(config_path)
            # self.config = full_config.get('GraphConstructor', {})

            # Hardcode fallback nếu không có file config thực tế
            self.re_model_name = "Qwen/Qwen2.5-3B-Instruct"
            self.llm = LLM(model_id=self.re_model_name)

        self.graph = nx.DiGraph()

    def _clean_text(self, text):
        return text.strip().replace('"', "'")

    def _parse_json_response(self, text: str) -> List[dict]:
        # [LOGIC CŨ GIỮ NGUYÊN]
        try:
            data = json.loads(text)
        except json.JSONDecodeError:
            try:
                match = re.search(r"```(?:json)?(.*?)```", text, re.DOTALL)
                if match:
                    json_str = match.group(1).strip()
                else:
                    match = re.search(r"\{.*\}", text, re.DOTALL)
                    if match:
                        json_str = match.group(0).strip()
                    else:
                        return []
                data = json.loads(json_str)
            except:
                return []

        if "graph" in data and isinstance(data["graph"], list):
            return data["graph"]
        if isinstance(data, list):
            return data
        return []

    # [MODIFIED] Thay thế _extract_e2e đơn lẻ bằng xử lý danh sách inputs
    def _extract_batch_e2e(self, texts: List[str]) -> List[List[dict]]:
        # Chuẩn bị inputs đúng format cũ
        user_inputs = [f"Analyze this Vietnamese text and output JSON only:\n\"{text}\"" for text in texts]

        try:
            # Gọi hàm generate_batch của class LLM
            response_contents = self.llm.generate_batch(
                inputs=user_inputs,
                system_prompt=self.E2E_PROMPT
            )

            # Parse từng kết quả trả về
            parsed_results = []
            for content in response_contents:
                parsed_results.append(self._parse_json_response(content))
            return parsed_results

        except Exception as e:
            print(f"   ❌ [Batch Extraction Error] {e}")
            # Trả về list rỗng tương ứng với số lượng input nếu lỗi
            return [[] for _ in texts]

    def _build_graph_from_triples(self, relations: List[dict]) -> nx.DiGraph:
        # [LOGIC CŨ GIỮ NGUYÊN]
        G = nx.DiGraph()
        if not relations: return G
        for item in relations:
            head = item.get("head")
            tail = item.get("tail")
            rel_label = item.get("relation")
            if not head or not tail or not rel_label: continue

            head = str(head).strip()
            tail = str(tail).strip()

            G.add_node(head, **item.get("head_props", {}))
            G.add_node(tail, **item.get("tail_props", {}))

            edge_attrs = {"relation": rel_label}
            edge_attrs.update(item.get("relation_props", {}))
            G.add_edge(head, tail, **edge_attrs)
        return G

    def process(self, data_input, text_column=None, verbose=True, batch_size=256):
        """
        [MODIFIED] Sửa logic loop để chạy theo batch_size
        """
        # 1. Chuẩn bị dữ liệu đầu vào (Giữ logic cũ)
        if isinstance(data_input, pd.DataFrame):
            data = data_input[text_column].tolist()
        elif isinstance(data_input, pd.Series):
            data = data_input.tolist()
        elif isinstance(data_input, list):
            data = data_input
        else:
            data = []

        list_of_graphs = []
        extraction_history = []

        # Xử lý text rỗng hoặc None trước để tránh lỗi model
        processed_data = [str(x) if x else "" for x in data]
        total_samples = len(processed_data)

        # 2. Vòng lặp Batch (Thay cho vòng lặp đơn)
        desc_text = f"Building Graphs (Batch Size={batch_size})"

        for i in tqdm(range(0, total_samples, batch_size), desc=desc_text, disable=not verbose):
            # Cắt batch
            batch_texts = processed_data[i : i + batch_size]

            # Gọi hàm xử lý batch mới
            batch_triples = self._extract_batch_e2e(batch_texts)

            # Lưu kết quả lại (Mapping 1-1 với input)
            for text, triples in zip(batch_texts, batch_triples):
                # Xây đồ thị cho từng mẫu (Logic cũ)
                local_graph = self._build_graph_from_triples(triples)
                list_of_graphs.append(local_graph)
                extraction_history.append({"input": text, "triples": triples})

        return list_of_graphs, extraction_history

## 4. Pipeline Setup

### 4.1. Dataset loading

In [13]:
def load_dataset():
    print('> Start loading dataset...')

    file_ids = {
        'train': '1s7kEVugop7Gv_iws5Kv7OyvLnoRVzjCA',
        'dev':   '1IrCff4cReAbdonve5__9fR3FC105d-LP',
        'test':  '1NPWG2KVeKQ6FPck851wCobPxDJIU3l5_'
    }

    base_url = 'https://drive.google.com/uc?id='
    dfs = {}
    try:
        for name, file_id in file_ids.items():
            dl_url = f"{base_url}{file_id}"
            print(f'  - Downloading & Reading {name} data...')
            dfs[name] = pd.read_csv(dl_url)

        print('> ✅ Dataset loaded successfully.')
        return dfs['train'], dfs['dev'], dfs['test']

    except Exception as e:
        print(f"\n❌ Lỗi khi tải dữ liệu: {e}")
        print("Mẹo: Hãy chắc chắn rằng link đã bật chế độ 'Anyone with the link' (Public).")
        return None, None, None

In [14]:
train, dev, test = load_dataset()

> Start loading dataset...
  - Downloading & Reading train data...
  - Downloading & Reading dev data...
  - Downloading & Reading test data...
> ✅ Dataset loaded successfully.


### 4.2. Main pipeline

In [17]:
import os
import pandas as pd
from tqdm import tqdm
from networkx.readwrite import json_graph

def main():
    # ---------------------------------------------------------
    # 1. SETUP: Khởi tạo Model & Graph Engine (Chỉ 1 lần)
    # ---------------------------------------------------------
    # Khởi tạo LLM A100 Mode (Native Bfloat16 + Flash Attention)
    print(">> 🚀 Initializing LLM Engine for A100...")
    llm_engine = LLM(model_id="Qwen/Qwen2.5-3B-Instruct")

    # Tạo config file nếu chưa có (Logic cũ)
    if not os.path.exists('configs/graph_constructor.yml'):
        os.makedirs('configs', exist_ok=True)
        with open('configs/graph_constructor.yml', 'w') as f:
            f.write("GraphConstructor:\n  RE_MODEL_NAME: 'Qwen/Qwen2.5-3B-Instruct'")

    # Truyền LLM engine vào để không phải load lại
    kg_processor = GraphConstructor(
        config_path='configs/graph_constructor.yml',
        llm_instance=llm_engine
    )

    # ---------------------------------------------------------
    # 2. LOAD DATA
    # ---------------------------------------------------------
    train, dev, test = load_dataset()

    if train is None:
        print("❌ Không có dữ liệu để chạy.")
        return

    datasets = [
        # ("dev", dev),
        # ("test", test),
        ("train", train),
    ]

    # Cấu hình Batch Size cho A100 (32-64 là lý tưởng cho model 3B)
    BATCH_SIZE = 128

    # ---------------------------------------------------------
    # 3. PROCESSING LOOP
    # ---------------------------------------------------------
    for name, data in datasets:
        print('=' * 50)
        print(f"Processing dataset: [{name}]")
        print(f'Shape: {data.shape}')
        print('=' * 50)

        # --- DEBUG MODE (Giữ logic cũ: Chạy thử vài dòng) ---
        # Bỏ comment dòng dưới để test nhanh (Chỉ chạy 1 batch đầu tiên)
        # data = data.head(5)

        result = {
            'id': [],
            'context': [],
            'prompt': [],
            'response': [],
            'label': [],
        }

        # Chia dataframe thành các chunk để xử lý batch mà không tràn RAM
        # Thay vì iterrows, ta loop theo index nhảy cóc (0, 32, 64...)
        total_rows = len(data)

        for start_idx in tqdm(range(0, total_rows, BATCH_SIZE), desc=f"Batch Run {name}"):
            # Lấy 1 chunk dữ liệu (ví dụ 32 dòng)
            end_idx = min(start_idx + BATCH_SIZE, total_rows)
            batch_df = data.iloc[start_idx : end_idx]

            # Chuẩn bị list text
            # Lưu ý: fillna("") để tránh lỗi model chết khi gặp NaN
            contexts = batch_df['context'].fillna("").tolist()
            prompts  = batch_df['prompt'].fillna("").tolist()
            responses = batch_df['response'].fillna("").tolist()

            # --- Extract KG (Batch Inference) ---
            # Gọi hàm process của kg_processor (đã sửa ở bước trước)
            # Hàm này giờ nhận vào LIST và trả về LIST Graphs

            # 1. Batch Context
            g_contexts, _ = kg_processor.process(contexts, batch_size=BATCH_SIZE, verbose=False)

            # 2. Batch Prompt
            g_prompts, _ = kg_processor.process(prompts, batch_size=BATCH_SIZE, verbose=False)

            # 3. Batch Response
            g_responses, _ = kg_processor.process(responses, batch_size=BATCH_SIZE, verbose=False)

            # --- Post-process & Store ---
            # Ghép kết quả lại vào result dict (Logic cũ: node_link_data)
            ids = batch_df.get('id', range(start_idx, end_idx)).tolist()
            labels = batch_df.get('label', [0]*len(batch_df)).tolist()

            for i in range(len(batch_df)):
                result['id'].append(ids[i])
                result['label'].append(labels[i])

                # Convert NetworkX -> JSON Dict (Logic cũ)
                result['context'].append(json_graph.node_link_data(g_contexts[i]))
                result['prompt'].append(json_graph.node_link_data(g_prompts[i]))
                result['response'].append(json_graph.node_link_data(g_responses[i]))

            #if start_idx > 4: break

        # ---------------------------------------------------------
        # 4. SAVE RESULTS
        # ---------------------------------------------------------
        output_dir = 'data/processed'
        os.makedirs(output_dir, exist_ok=True)

        output_path = f'{output_dir}/graph_{name}.csv'
        print(f'\n>> Saving {name} graph data to {output_path}...')

        result_df = pd.DataFrame(result)
        result_df.to_csv(output_path, index=False)
        print("Done.")

    # ---------------------------------------------------------
    # 5. CLEANUP & DOWNLOAD
    # ---------------------------------------------------------
    # Giải phóng VRAM
    if hasattr(kg_processor, 'llm'):
        kg_processor.llm.unload_model()

    print("\n>> 📦 Zipping and downloading results...")
    try:
        from google.colab import files
        os.system("zip -r processed_data.zip data/processed")
        files.download('processed_data.zip')
    except ImportError:
        print("⚠️ Không phải môi trường Colab, file đã được lưu tại folder data/processed.")

    return

In [18]:
main()

>> 🚀 Initializing LLM Engine for A100...
>> 📥 Loading Model: Qwen/Qwen2.5-3B-Instruct...
>> 🚀 Hardware Optimization: A100 Mode (Flash Attention 2 + bfloat16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

>> ✅ Model Loaded Successfully!
>> 💾 VRAM Used: 13.89 GiB
> Start loading dataset...
  - Downloading & Reading train data...
  - Downloading & Reading dev data...
  - Downloading & Reading test data...
> ✅ Dataset loaded successfully.
Processing dataset: [train]
Shape: (5600, 5)


Batch Run train: 100%|██████████| 44/44 [2:29:59<00:00, 204.53s/it]



>> Saving train graph data to data/processed/graph_train.csv...
Done.
>> 🗑️ Unloading model Qwen/Qwen2.5-3B-Instruct...
>> ✅ Memory cleared!

>> 📦 Zipping and downloading results...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>